# Time correlation on PWV and Discrete Covariance Function

- author Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date 2024-11-08 :
- update 2025-02-05 

- Kernel @usdf **w_2025_50*
- Office emac : mamba_py311
- Home emac : base (conda)
- laptop : conda_py310
- pyzdcf : readthedoc , https://pyzdcf.readthedocs.io/en/latest/ and github, https://github.com/LSST-sersag/pyzdcf, availle in pypi : https://pypi.org/project/pyzdcf/
- CANNOT INSTALL PYZDCF AT USDF

**Goal** : Show correlation holo /Merra

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os

In [ ]:
from pyzdcf import pyzdcf

In [ ]:
# where are stored the figures
pathfigs = "figsHoloCorrelationPWVTime-v2"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
pathdata = "dataHoloCorrelationPWVTime"
if not os.path.exists(pathdata):
    os.makedirs(pathdata) 
datapath_input = os.path.join(pathdata,"timecurves") 
datapath_output = os.path.join(pathdata,"results") 
if not os.path.exists(datapath_input):
    os.makedirs(datapath_input) 
if not os.path.exists(datapath_output):
    os.makedirs(datapath_output) 

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from astropy.coordinates.earth import EarthLocation
from datetime import datetime
from pytz import timezone

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit,least_squares


# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# Remove to run faster the notebook
#import ipywidgets as widgets
#%matplotlib widget

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

## Configuration

In [ ]:
observing_location = EarthLocation.of_site('Rubin Observatory')
tz = timezone('America/Santiago')

### Spectro Hologram data

In [ ]:
FLAG_WITHCOLLIMATOR = False
DATE_WITHCOLLIMATOR = 20230930
datetime_WITHCOLLIMATOR = convertNumToDatestr(DATE_WITHCOLLIMATOR)
datetime_WITHCOLLIMATOR = pd.to_datetime("2023-09-30 00:00:00.0+0000")
datetime_WITHCOLLIMATOR

In [ ]:
version_results = "v5"
legendtag = {"v1" : "old v3.1.0",
            "v2" : "v3.1.0-PWV<10mm",
            "v3" : "v3.1.0-PWV<15mm",
            "v4" : "Auxtel holo v3.1.0",
            "v5" : "Auxtel holo v3.1.0 09/22 - 10/24"}

In [ ]:
atmfilenamesdict = {"v1" : "data/spectro/auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_testWithMaskedEdges_newBoundaries_newPolysRescaled_newFitBounds_adjustA1_lockedOrder2_removeThroughputTails_2.npy",
                    "v2" : "auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_PeekFinder.npy",
                    "v3" : "u_dagoret_auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_PeekFinder_20240924T161119Z.npy",
                    "v4" : "u_dagoret_auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_PeekFinder_20240924T161119Z_spectrfullextend.npy",
                    "v5" : "u_dagoret_auxtel_atmosphere_202209_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_No5SigmaClip_20241016T184601Z_spectrfullextended.npy"}

In [ ]:
atmfilename = atmfilenamesdict[version_results]
tag = legendtag[version_results] 

## Initialisation

### Read the file

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)

In [ ]:
list(df_spec.columns)

### Remove spectra with red filter

In [ ]:
df_spec['FILTER'].unique()

In [ ]:
FLAG_REMOVE_FILTERS = True
if FLAG_REMOVE_FILTERS:
    df_spec=df_spec[df_spec["FILTER"] == 'empty']
    df_spec.reset_index(inplace=True)  

### Define if a target is faint or bright

In [ ]:
def IsFaint(row):
    List_Of_Faint_targets = ['Feige110','HD074000','HD115169','HD031128','HD200654','HD167060','HD009051','HD142331','HD160617','HD111980']
    List_Of_faint_selected = List_Of_Faint_targets[:10]
    if row["TARGET"] in List_Of_faint_selected:
        return True
    else:
        return False

In [ ]:
df_spec["isFaint"] = df_spec.apply(IsFaint,axis=1)

### Compute NightObs

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
if FLAG_WITHCOLLIMATOR:
    df_spec = df_spec[df_spec["nightObs"]> DATE_WITHCOLLIMATOR]

## Apply Quality selection

In [ ]:
fig,ax = plt.subplots(1,1)
df_spec["CHI2_FIT"].hist(bins=50,ax=ax,range=(0,200))
ax.set_yscale("log")

### Add the Time in pd.datetime

#### UTC

In [ ]:
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])

In [ ]:
DT = pd.Timedelta(minutes=7*24*60)
TMIN  = df_spec["Time"].min()-DT
TMAX  = df_spec["Time"].max()+DT

### Compute relative time to Mid-night

In [ ]:
def GetTimeToMidNight(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(local_time.year,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
    local_time_midnight = datetime(local_time_new.year,local_time_new.month,local_time_new.day)
    dt_hour = (local_time_new -local_time_midnight).seconds/3600.

    # we took the previous night mid-night , must subtract 24H
    if dt_hour > 12.:
        dt_hour_new = (dt_hour - 24.)
    else:
        dt_hour_new = dt_hour
        
    return dt_hour_new

In [ ]:
df_spec["dt_midnight"] = df_spec.apply(GetTimeToMidNight,axis=1)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,4))
df_spec["dt_midnight"].hist(bins=48,range=(-12,12),ax=ax,facecolor="blue") 
ax.set_xlabel("time relative to midnight (hour)")
ax.set_title("Observation time")

### Compute Date relative to January

In [ ]:
def GetDateToMidJanuary(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(2024,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
           
    return pd.to_datetime(local_time_new)

In [ ]:
#df_spec["Time_january"] = df_spec.apply(GetDateToMidJanuary,axis=1)

In [ ]:
def GetDateToMidJanuaryAndYear(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(2024,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
           
    return pd.to_datetime(local_time_new),local_time.year 

In [ ]:
df_spec[["Time_january","Year"]] = df_spec.apply(GetDateToMidJanuaryAndYear,axis=1,result_type="expand")

In [ ]:
df_spec[["Time_january","Year"]]

## Compute night boundaries

In [ ]:
def GetNightBoundariesDict(df_spec):
    """
    input:
      df_spec the dataframe for spectroscopy summary results
    output:
      the dict of night boudaries
    """
    
    Dt = pd.Timedelta(minutes=30)
    d = {}
    list_of_nightobs = df_spec["nightObs"].unique()
    for nightobs in list_of_nightobs:
        sel_flag = df_spec["nightObs"]== nightobs
        df_night = df_spec[sel_flag]
        tmin = df_night["Time"].min()-Dt
        tmax = df_night["Time"].max()+Dt
        d[nightobs] = (tmin,tmax)
    return d

In [ ]:
dn = GetNightBoundariesDict(df_spec)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()

ax.set_xlim(TMIN,TMAX) 
#df_m.plot(x="Time",y="PShP",ax=ax,marker=".",c="b",lw=0.5,label="Merra2",ms=1,legend=leg)   
df_spec.plot(x="Time",y="OUTPRESS",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)
ax.set_ylabel("OUTPRESS")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Pressure (before quality cuts)")
ax.legend()

if not FLAG_WITHCOLLIMATOR:
    ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)


for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)


#ax.set_ylim(0.,15.)

figname =f"{pathfigs}/pressure_allpoints_allnights_nocuts"+figtype
plt.savefig(figname)
plt.show()


In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()

ax.set_xlim(TMIN,TMAX)   
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)
ax.set_ylabel("PWV [mm]_x")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Precipitable water vapor measured by holo vs time (before quality cuts)")
ax.legend()

if not FLAG_WITHCOLLIMATOR:
    ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)


for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)


ax.set_ylim(0.,15.)

figname =f"{pathfigs}/pwv_allpoints_allnights_nocuts"+figtype
plt.savefig(figname)
plt.show()


## Apply Quality selection cuts

In [ ]:
def getSelectionCut(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.):
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) 
    return cut

In [ ]:
def getSelectionCutNoPolar(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.):
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] != "HD185975")
    return cut

In [ ]:
def getSelectionCutWithPolar(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.):
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] == "HD185975")
    return cut

In [ ]:
cut = getSelectionCut(df_spec) 
cut_nopolar = getSelectionCutNoPolar(df_spec) 
cut_nopolar_bright = getSelectionCutNoPolar(df_spec) & (~df_spec["isFaint"])
cut_nopolar_faint = getSelectionCutNoPolar(df_spec) & (df_spec["isFaint"])
cut_wthpolar = getSelectionCutWithPolar(df_spec)

In [ ]:
df_spec_sel = df_spec[cut]
df_spec_np = df_spec[cut_nopolar] 
df_spec_np_b = df_spec[cut_nopolar_bright]
df_spec_np_f = df_spec[cut_nopolar_faint]
df_spec_wp = df_spec[cut_wthpolar]

In [ ]:
print("Total number of Spectra          : ",len(df_spec))
print("Number of selected Spectra       : ",len(df_spec_sel))
print("Number of selected Polars        : ",len(df_spec_wp))
print("Number of selected Non-Polars    : ",len(df_spec_np))
print("Number of selected Non-Polars Bright : ",len(df_spec_np_b))
print("Number of selected Non-Polars Faint  : ",len(df_spec_np_f))

In [ ]:
df_spec_sel.reset_index(drop=True,inplace=True)
df_spec_np.reset_index(drop=True,inplace=True)
df_spec_wp.reset_index(drop=True,inplace=True) 
df_spec_np_b.reset_index(drop=True,inplace=True)
df_spec_np_f.reset_index(drop=True,inplace=True)

In [ ]:
#List_Of_Faint_targets = ['Feige110','HD074000','HD115169','HD031128','HD200654','HD167060','HD009051','HD142331','HD160617','HD111980']
print("Polar            :",len(df_spec_wp["TARGET"].unique()),"\t", df_spec_wp["TARGET"].unique()) 
print("Non Polar        :",len(df_spec_np["TARGET"].unique()),"\t" ,df_spec_np["TARGET"].unique())
print("Non Polar Bright :",len(df_spec_np_b["TARGET"].unique()),"\t" ,df_spec_np_b["TARGET"].unique())
print("Non Polar Faint  :",len(df_spec_np_f["TARGET"].unique()),"\t",df_spec_np_f["TARGET"].unique())

## Recompute night boundaries

In [ ]:
#dn = GetNightBoundariesDict(df_spec_sel)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%d")

fig,axs = plt.subplots(1,1,figsize=(14,6),layout='constrained')
ax  = axs
leg=ax.get_legend()

#ax.set_xlim(TMIN,TMAX) 
#df_m.plot(x="Time",y="TQV",ax=ax,marker=".",c="b",lw=0.5,ms=1,label="Merra2",legend=leg)     
    
df_spec_sel[df_spec_sel.Year==2022].plot(x="Time_january",y="PWV [mm]_x",ax=ax,marker='+',c="g",lw=0.0,grid=True,label="2022",legend=leg,ms=5,alpha=1.)
df_spec_sel[df_spec_sel.Year==2023].plot(x="Time_january",y="PWV [mm]_x",ax=ax,marker='+',c="b",lw=0.0,grid=True,label="2023",legend=leg,ms=5,alpha=1.)
df_spec_sel[df_spec_sel.Year==2024].plot(x="Time_january",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label="2024",legend=leg,ms=5,alpha=1.)
#df_spec_np_f.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='o',c="grey",lw=0.0,grid=True,label="faint,no polar",legend=leg,ms=5,alpha=0.2)
#df_spec_np_b.plot(x="Time_january",y="PWV [mm]_x",ax=ax,marker='o',c="r",lw=0.0,grid=True,label="bright,no polar",legend=leg,ms=5,alpha=0.2)
#df_spec_np.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='o',c="m",lw=0.0,grid=True,label="no polar",legend=leg,ms=5,alpha=0.5)
#df_spec_wp.plot(x="Time_january",y="PWV [mm]_x",ax=ax,marker='o',c="g",lw=0.0,grid=True,label="with polar",legend=leg,ms=5,alpha=0.5)
ax.set_ylabel("PWV [mm]_x")
ax.set_xlabel("date (since January)")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Precipitable water vapor measured by holo (modulo 1 year)",fontweight="bold")
ax.legend()

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)

plt.tight_layout()
 
ax.set_ylim(0.,15.)
#ax.set_xlim(TMIN,TMAX)

figname =f"{pathfigs}/pwvholoM2_allpoints_allnights_modulo1year"+figtype
plt.savefig(figname)
plt.show()

## Plot night by night

In [ ]:
def ComputeRepeatability(df):
    """
    Compute Repeatability of PWV for Spectrogram and and Spectrum
    
    """
    N = len(df)
    dfout = pd.DataFrame(index=df.index,columns = ["nightObs","dt","dt_rep","dPWVx_rep","dPWVy_rep","targflag_rep","Npoints"])
    dfout["targflag_rep"].astype(bool)
    #dfout["Npoints"].astype(int)
    
    target_old = "No"
    time_old = 0.
    pwvx_old = 0.
    pwvy_old = 0.
    
    for index in range(N):
        
        nightObs =  df.iloc[index]["nightObs"]
        
        if index ==0:
            dt0 = df.iloc[index]["dt"]
            dfout.iloc[index] = [ nightObs,dt0, 0., 0., 0., False,N]
        else:
            target_new = df.iloc[index]["TARGET"]
            time_new = df.iloc[index]["dt"]
            pwvx_new = df.iloc[index]["PWV [mm]_x"]
            pwvy_new = df.iloc[index]["PWV [mm]_y"]
            
            flag_target = (target_new == target_old)
            dPWVx_rep = pwvx_new - pwvx_old
            dPWVy_rep = pwvy_new - pwvy_old
            dt_rep = (time_new-time_old)*3600. # in seconds
            
            dfout.iloc[index] = [ nightObs,time_new, dt_rep, dPWVx_rep, dPWVy_rep, flag_target,N]
        
        target_old = df.iloc[index]["TARGET"]
        time_old = df.iloc[index]["dt"]
        pwvx_old = df.iloc[index]["PWV [mm]_x"]
        pwvy_old = df.iloc[index]["PWV [mm]_y"]
        
    return dfout

In [ ]:
all_selected_nights = df_spec_sel["nightObs"].unique()

## Fits gaussien et lineaires

In [ ]:
def funclineres(params, x, y, yerr):
    # Return residual = fit-observed
    return (y-params[0] -params[1]*x)/yerr
def funcline(params,x):
    return params[0] + params[1]*x

In [ ]:
def MakeLineFit(df_night_pwv_curve):
    """
    """

    x = df_night_pwv_curve["dt_midnight"].values
    y = df_night_pwv_curve["PWV [mm]_x"].values
    yerr = df_night_pwv_curve["PWV [mm]_err_x"].values
    n = len(y)
        
    #popt, pcov = optimize.curve_fit(f, x, y, [1,-4])
    fit_res = least_squares(funclineres,[5.,0],args = (x,y,yerr))
    popt = fit_res.x 
    npar = len(popt)
    J = fit_res.jac
    cov = np.linalg.inv(J.T.dot(J))
    chi2dof = ((funclineres(popt,x,y,yerr))**2).sum()/(n-npar)
    cov *= chi2dof
    perr = np.sqrt(np.diagonal(cov)) 
    
    xfit = np.linspace(x.min()*0.99,x.max()*1.05)
    yfit = funcline(popt,xfit)
        
    slope = popt[1]
    slope_err = perr[1]

    return x,y,yerr,n,chi2dof,xfit,yfit,slope,slope_err

In [ ]:
#
# Gaussian Process regression
# https://scikit-learn.org/1.5/auto_examples/gaussian_process/plot_gpr_noisy_targets.html
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

In [ ]:
def MakeGaussianProcessFit(df_night_pwv_curve):
    """
    """

    x_train = df_night_pwv_curve["dt_midnight"].values
    xmin =  x_train.min()
    xmax = x_train.max()
    xfit = np.linspace(xmin,xmax,50)
    X_fit =xfit.reshape(-1,1)
    X_train = x_train.reshape(-1, 1)
    
    y_train = df_night_pwv_curve["PWV [mm]_x"].values
    y_mean = y_train.mean()
    yerr_train = df_night_pwv_curve["PWV [mm]_err_x"].values
    n = len(y_train)

    noise_std= 0.5
    
    kernel = 1. * RBF(length_scale=5.0, length_scale_bounds=(0.5, 12.))
    gaussian_process = GaussianProcessRegressor(kernel=kernel,alpha=noise_std**2 ,n_restarts_optimizer=9)
    
    gaussian_process.fit(X_train, y_train)

    mean_prediction, std_prediction = gaussian_process.predict(X_fit, return_std=True)
    return xfit, mean_prediction, std_prediction, gaussian_process

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
# https://scikit-learn.org/1.5/auto_examples/linear_model/plot_ard.html
from sklearn.linear_model import ARDRegression,BayesianRidge

In [ ]:
POLY_DEGREE_MAX = 10

In [ ]:
def MakeLinearModelFit(df_night_pwv_curve,degree_max = POLY_DEGREE_MAX ):
    """
    https://scikit-learn.org/1.5/auto_examples/linear_model/plot_ard.html
    ARDRegression and BayesianRidge
    """

    x_train = df_night_pwv_curve["dt_midnight"].values
    xmin =  x_train.min()
    xmax = x_train.max()
    xfit = np.linspace(xmin,xmax,50)
    X_fit =xfit.reshape(-1,1)
    X_train = x_train.reshape(-1, 1)
    
    y_train = df_night_pwv_curve["PWV [mm]_x"].values
    y_mean = y_train.mean()
    yerr_train = df_night_pwv_curve["PWV [mm]_err_x"].values
    n = len(y_train)

    ard_poly = make_pipeline(
    PolynomialFeatures(degree=degree_max, include_bias=False),StandardScaler(),ARDRegression(),).fit(X_train, y_train)
    
    brr_poly = make_pipeline(
    PolynomialFeatures(degree=degree_max, include_bias=False),StandardScaler(),BayesianRidge(),).fit(X_train, y_train)

    y_ard, y_ard_std = ard_poly.predict(X_fit, return_std=True)
    y_brr, y_brr_std = brr_poly.predict(X_fit, return_std=True)

    y_ard_pred = ard_poly.predict(X_train,return_std=False)
    resy = y_train - y_ard_pred
    
    return xfit,y_ard, y_ard_std, y_brr, y_brr_std, resy, y_mean

In [ ]:
# Container of images that all goes in a pdf
all_figs_to_pdf = []

In [ ]:
all_dateObs_sel = {}

# loop on nights
for night in all_selected_nights:
    #select the night
    # Choose the kind of observation
    df_spec_night = df_spec_sel[df_spec_sel["nightObs"] == night]
    df_spec_night_wp = df_spec_wp[df_spec_wp["nightObs"] == night]
    df_spec_night_np = df_spec_np[df_spec_np["nightObs"] == night]
    df_spec_night_np_f = df_spec_np_f[df_spec_np_f["nightObs"] == night]
    df_spec_night_np_b = df_spec_np_b[df_spec_np_b["nightObs"] == night]
    
    #select the variables
    df_night_pwv_curve = df_spec_night[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_wp = df_spec_night_wp[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_np_b = df_spec_night_np_b[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_np_f = df_spec_night_np_f[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_np = df_spec_night_np[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]

    tmin = df_night_pwv_curve["Time"].min()
    tmax = df_night_pwv_curve["Time"].max()

    list_of_targets = df_night_pwv_curve["TARGET"].unique()
    str_list_of_targets = "\n".join(list_of_targets)
     
    # convert in hours
    df_night_pwv_curve["dt"] = (df_night_pwv_curve["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_wp["dt"] = (df_night_pwv_curve_wp["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_np["dt"] = (df_night_pwv_curve_np["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_np_f["dt"] = (df_night_pwv_curve_np_f["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_np_b["dt"] = (df_night_pwv_curve_np_b["Time"] - tmin).dt.total_seconds()/3600.
    
    # extract statistics on pwv
    stat = df_night_pwv_curve[["PWV [mm]_x"]].describe()
    date_form = DateFormatter("%y-%m-%dT%H:%M")
    #date_form = DateFormatter("%y-%m-%dT%H")
    count = int(stat.loc["count"].values[0])
    mean = stat.loc["mean"].values[0]
    median = stat.loc["50%"].values[0]
    std = stat.loc["std"].values[0]
    textstr = "\n".join((f"count : {count}",
                     f"mean : {mean:.1f} mm",
                     f"median : {median:.1f} mm",
                     f"std : {std:.1f} mm",
                    ))


    N= len(df_night_pwv_curve)

    if N>10:

        if 0:
        
            # does the fit
        
            #x,y,yerr,n,chi2dof,xfit,yfit,slope,slope_err = MakeLineFit(df_night_pwv_curve)
            
            xfit, mean_prediction, std_prediction, gp = MakeGaussianProcessFit(df_night_pwv_curve)
            xfit2, y_ard, y_ard_std, y_brr, y_brr_std, resy, y_mean = MakeLinearModelFit(df_night_pwv_curve)
        
       
    
        
            # plot

            #layout="constrained"
            fig = plt.figure(figsize=(16,4),constrained_layout=True)
            gs = GridSpec(1, 2,  width_ratios=[1,1],figure=fig)
    
            ax = fig.add_subplot(gs[0])
            ax2 = fig.add_subplot(gs[1])
        
            leg=ax.get_legend()
            leg2=ax2.get_legend()
      
        
            ################################
            # left figure : Gaussian Process
            ################################
        
            ax.errorbar(x=df_night_pwv_curve.dt_midnight, y=df_night_pwv_curve["PWV [mm]_x"],yerr=df_night_pwv_curve["PWV [mm]_err_x"],fmt="+",color="r",ecolor="k",ms=1)
            df_night_pwv_curve_wp.plot(x="dt_midnight",y="PWV [mm]_x",c="cyan",ax=ax,marker='o',lw=0.0,grid=True,legend=leg2,label="polar",ms=6,alpha=1)
            df_night_pwv_curve_np_b.plot(x="dt_midnight",y="PWV [mm]_x",c="r",ax=ax,marker='o',lw=0.0,grid=True,legend=leg2,label="bright, no polar",ms=6,alpha=1)
            df_night_pwv_curve_np_f.plot(x="dt_midnight",y="PWV [mm]_x",c="g",ax=ax,marker='o',lw=0.0,grid=True,legend=leg2,label="faint, no polar",ms=6,alpha=1)

            # fit result
            ax.plot(xfit, mean_prediction,"b-",lw=3,label="Gaussian Process (RBF)")
            ax.fill_between(xfit,mean_prediction - 1.96 * std_prediction,mean_prediction + 1.96 * std_prediction,alpha=0.3,label=r"95% CL",facecolor='grey')
        
            ax.set_xlabel("hours")
            ax.set_ylim(0.,15.)
            ax.legend(loc="upper right")
        
            ax.set_title(f"GP fit for night {night}")
            ax.axvline(x=0,color="k",lw=2)


            ############################
            # right figure : linear fits
            ############################
            ax2.errorbar(x=df_night_pwv_curve.dt_midnight, y=df_night_pwv_curve["PWV [mm]_x"],yerr=df_night_pwv_curve["PWV [mm]_err_x"],fmt="+",color="r",ecolor="k",ms=1)
            df_night_pwv_curve_wp.plot(x="dt_midnight",y="PWV [mm]_x",c="cyan",ax=ax2,marker='o',lw=0.0,grid=True,legend=leg2,label="polar",ms=6,alpha=1)
            df_night_pwv_curve_np_b.plot(x="dt_midnight",y="PWV [mm]_x",c="r",ax=ax2,marker='o',lw=0.0,grid=True,legend=leg2,label="bright, no polar",ms=6,alpha=1)
            df_night_pwv_curve_np_f.plot(x="dt_midnight",y="PWV [mm]_x",c="g",ax=ax2,marker='o',lw=0.0,grid=True,legend=leg2,label="faint, no polar",ms=6,alpha=1)


            ax2.plot(xfit2, y_ard,"g-",lw=3,label="ARDRegression")
            ax2.fill_between(xfit2, y_ard - 1.96 * y_ard_std,y_ard + 1.96 * y_ard_std,alpha=0.3,label=r"95% CL",facecolor='green')

        

        
            ax2.set_title(f"Linear fit for night {night}")
            ax2.set_xlabel("hours")
            ax2.set_ylim(0.,15.)
            ax2.legend()
            ax2.axvline(x=0,color="k",lw=2)
        
        

        

            figname =f"{pathfigs}/gp_ard_pwv_per_night_{night}"+figtype
            plt.savefig(figname)
            plt.show()

        # add statistics
        all_dateObs_sel[night] = stat
        

## PWV difference and Time difference

In [ ]:
def ComputePWVAndTimeDiffence(df):
    """
    For each measuremet retrun the delta pwv nd delta_t with next measurements

    """
    all_DPWV = []
    all_DT = []
    all_pwvpwv = []
    
    N = len(df)

    meanPWV = df.describe().loc["mean"]["PWV [mm]_x"] 
        
    for row1 in df.iterrows(): 
        t1 = row1[1]["dt"]
        PWV1 = row1[1]["PWV [mm]_x"]
        pwv1 = PWV1 - meanPWV
        for row2 in df.iterrows(): 
            t2 = row2[1]["dt"]
            PWV2 = row2[1]["PWV [mm]_x"]
            pwv2 = PWV2 - meanPWV
            Delta_t = t2-t1
            Delta_pwv = PWV2-PWV1
            pwv1pwv2 = pwv1*pwv2
            if  Delta_t>0:
                all_DPWV.append(Delta_pwv)
                all_DT.append(Delta_t)
                all_pwvpwv.append(pwv1pwv2)
    return np.array(all_DT), np.array(all_DPWV), np.array(all_pwvpwv)  

In [ ]:
df_night_pwv_curve.describe().loc["mean"]["PWV [mm]_x"] 

In [ ]:
sigma_repeatability = 0.26

## Plot the time behaviour

In [ ]:
# container of all quantities
all_sigmapwv = []
all_meanpwv = []
all_TimeDifferences = [] # t2-t1
all_PWVDifferences = [] # PWV2-PWV1
all_PWVProducts = [] # pwvi x pwvj
all_nights_forpairs = []


# loop on nights
for night in all_selected_nights:
    #select the night
    # Choose the kind of observation
    df_spec_night = df_spec_sel[df_spec_sel["nightObs"] == night]
    df_spec_night_wp = df_spec_wp[df_spec_wp["nightObs"] == night]
    df_spec_night_np = df_spec_np[df_spec_np["nightObs"] == night]
    df_spec_night_np_f = df_spec_np_f[df_spec_np_f["nightObs"] == night]
    df_spec_night_np_b = df_spec_np_b[df_spec_np_b["nightObs"] == night]
    
    #select the variables
    df_night_pwv_curve = df_spec_night[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_wp = df_spec_night_wp[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_np_b = df_spec_night_np_b[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_np_f = df_spec_night_np_f[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_np = df_spec_night_np[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]

    tmin = df_night_pwv_curve["Time"].min()
    tmax = df_night_pwv_curve["Time"].max()

    list_of_targets = df_night_pwv_curve["TARGET"].unique()
    str_list_of_targets = "\n".join(list_of_targets)
    str_list_of_targets = "\n".join(["targets:",str_list_of_targets])
     

    # convert in hours
    df_night_pwv_curve["dt"] = (df_night_pwv_curve["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_wp["dt"] = (df_night_pwv_curve_wp["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_np["dt"] = (df_night_pwv_curve_np["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_np_f["dt"] = (df_night_pwv_curve_np_f["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_np_b["dt"] = (df_night_pwv_curve_np_b["Time"] - tmin).dt.total_seconds()/3600.
    
    # extract statistics on pwv
    stat = df_night_pwv_curve[["PWV [mm]_x"]].describe()
    date_form = DateFormatter("%y-%m-%dT%H:%M")
    #date_form = DateFormatter("%y-%m-%dT%H")
    count = int(stat.loc["count"].values[0])
    mean = stat.loc["mean"].values[0]
    median = stat.loc["50%"].values[0]
    std = stat.loc["std"].values[0]
    textstr = "\n".join((f"count : {count}",
                     f"mean : {mean:.1f} mm",
                     f"median : {median:.1f} mm",
                     f"std : {std:.1f} mm",
                    ))


    N= len(df_night_pwv_curve)

    if N>10:
        # does the fit
        
        #x,y,yerr,n,chi2dof,xfit,yfit,slope,slope_err = MakeLineFit(df_night_pwv_curve)
        xfit, y_ard, y_ard_std, y_brr, y_brr_std, resy, y_mean = MakeLinearModelFit(df_night_pwv_curve)
        all_dt,all_dpwv,all_pwvpwv = ComputePWVAndTimeDiffence(df_night_pwv_curve)
        sigma_of_the_night = np.std(resy)


        all_nights_forpairs.append(night)
        
        # Fill t,pwv
        all_TimeDifferences.append(all_dt)
        all_PWVDifferences.append(all_dpwv)
        all_PWVProducts.append(all_pwvpwv)
            
        #mean and std of the night
        all_meanpwv.append(y_mean)
        all_sigmapwv.append(sigma_of_the_night)

        
        #textstr2 = "\n".join((f"d(PWV/dt) : ",
        #             f"slope : {slope:.3f} mm/h",
        #             f"slope err : {slope_err:.3f} mm/h",
        #             f"chi2/ndeg : {chi2dof:.2f}",       
        #            ))

        #textstr3 = "\n".join((f"d(PWV/dt) : ",
        #             f"slope : {slope:.3f} mm/h",
        #             f"slope err : {slope_err:.3f} mm/h",  
        #            ))
    

        #stat.loc["slope","PWV [mm]_x"] = slope 
        #stat.loc["slope_err","PWV [mm]_x"] = slope_err 
        #stat.loc["chi2","PWV [mm]_x"] = chi2dof

        ##########
        # plot
        ############
        

        fig = plt.figure(figsize=(18,4),constrained_layout=True)
        gs = GridSpec(1, 3,  width_ratios=[2,1,1],figure=fig)
    
        ax = fig.add_subplot(gs[0])
        ax2 = fig.add_subplot(gs[1])
        ax3 = fig.add_subplot(gs[2])
        
        leg=ax.get_legend()
        leg2=ax2.get_legend()
        leg3=ax3.get_legend()
        
        ax.errorbar(x=df_night_pwv_curve.dt_midnight, y=df_night_pwv_curve["PWV [mm]_x"],yerr=df_night_pwv_curve["PWV [mm]_err_x"],fmt="+",color="r",ecolor="k",label="stat err",ms=1)
        df_night_pwv_curve_wp.plot(x="dt_midnight",y="PWV [mm]_x",c="cyan",ax=ax,marker='o',lw=0.0,grid=True,legend=leg,label="polar",ms=6,alpha=1)
        df_night_pwv_curve_np_b.plot(x="dt_midnight",y="PWV [mm]_x",c="r",ax=ax,marker='o',lw=0.0,grid=True,legend=leg,label="bright, no polar",ms=6,alpha=1)
        df_night_pwv_curve_np_f.plot(x="dt_midnight",y="PWV [mm]_x",c="g",ax=ax,marker='o',lw=0.0,grid=True,legend=leg,label="faint, no polar",ms=6,alpha=1)
        ax.set_xlabel("hours since midnight")
        ax.set_ylim(0.,15.)
        ax.set_ylabel("PWV (mm)")
        ax.legend(loc="upper right")

        ax.plot(xfit, y_ard,"g-",lw=3,label="ARDRegression")
        ax.fill_between(xfit, y_ard - 1.96 * y_ard_std,y_ard + 1.96 * y_ard_std,alpha=0.3,label=r"95% CL",facecolor='green')

        #ax.plot(xfit2, y_brr,"r-",label="BayesianRidge")
        #ax.fill_between(xfit2, y_brr - 1.96 * y_brr_std,y_brr + 1.96 * y_brr_std,alpha=0.5,label=r"95% CL",facecolor='red')
      
       
        ax.text(0.8, 0.5, str_list_of_targets, transform=ax.transAxes, fontsize=12,verticalalignment='top', bbox=props,alpha=0.5)
        ax.set_title(f"night {night}")
        ax.axvline(x=0,color="k",lw=2)


        inset_ax = inset_axes(ax,width="20%", # width = 30% of parent_bbox
                                          height="40%", # height : 1 inch
                                          loc="upper left")
        
        textstr_sigma = f"$\sigma = $" +f"{sigma_of_the_night:.2f} mm"
        inset_ax.hist(resy,bins=30,facecolor="b")
        inset_ax.set_xlabel("residuals (mm)")
        inset_ax.text(0.1, 0.9, textstr_sigma , transform=inset_ax.transAxes, fontsize=8,verticalalignment='top', bbox=props,alpha=1)



        ax2.scatter(all_dt,np.abs(all_dpwv),marker="o",c="b",s=20,alpha=.5)
        ax2.set_xlabel("$\Delta t$ (hours)")
        ax2.set_ylabel("$|\Delta PWV |$ (mm)")
        ax2.grid()
        ax2.set_title("PWV difference and time difference")
        #ax2.axhspan(0, 2*sigma_repeatability, facecolor="grey", alpha=0.8,label="95 CL PWV repeatability")
        ax2.set_xscale("log")
        ax2.legend()

        ax3.scatter(all_dt,all_pwvpwv/sigma_repeatability**2,marker="o",c="r",s=20,alpha=.5)
        ax3.set_xlabel("$\Delta t$ (hours)")
        ax3.set_ylabel("$ dpwv1 \cdot dpwv2/\sigma^2$")
        ax3.grid()
        ax3.set_title("$ dpwv1 \cdot dpwv2/\sigma^2 \; vs\; \delta t$")
        #ax2.axhspan(0, 2*sigma_repeatability, facecolor="grey", alpha=0.8,label="95 CL PWV repeatability")
        ax3.set_xscale("log")
        ax3.legend()


        
        all_figs_to_pdf.append(fig) 
        
        plt.tight_layout()
        figname =f"{pathfigs}/pwvtimecorrelation_per_night_{night}"+figtype
        plt.savefig(figname)
        plt.show()


## Make a pandas dataframe with all time and pwv differences

In [ ]:
# loop on nights
df_pairs = pd.DataFrame(columns = ["idxnight","idxpair","nightobs","meanpwv","sigmapwv","dt","dPwv","PwvixPwvj"])
ientry = 0

for idxnight, night in enumerate(all_nights_forpairs):
    the_sigmapwv = all_sigmapwv[idxnight]
    the_meanpwv = all_meanpwv[idxnight]
    the_TimeDifferences = all_TimeDifferences[idxnight]
    the_PWVDifferences = all_PWVDifferences[idxnight]
    the_PWVProducts = all_PWVProducts[idxnight]

    Npairs = len(the_TimeDifferences)

    # loop on pairs 
    for idxpair in range(Npairs):
        df_pairs.loc[ientry] = [idxnight,idxpair, night,the_meanpwv , the_sigmapwv, the_TimeDifferences[idxpair], the_PWVDifferences[idxpair], the_PWVProducts[idxpair]  ]
        ientry+=1

In [ ]:
df_pairs = df_pairs.astype({"idxnight":"int","idxpair":"int","nightobs":"int"})

In [ ]:
df_pairs

## Stat plot 

In [ ]:
import seaborn as sns

In [ ]:
sns.violinplot(x=df_pairs["dt"])

In [ ]:
sns.violinplot(x=df_pairs["dPwv"])

### Generate time bins

In [ ]:
MAXHOURTIMES = 10.0    # hours
TIMEBINWIDTHMIN = 10.0 # in minutes
NUMBEROFTIMEBINS = int(MAXHOURTIMES*60.0/TIMEBINWIDTHMIN)

In [ ]:
def time_to_bintime(thour,binwidth=TIMEBINWIDTHMIN):
    """
    thour : time difference in hour
    binwidth : width of the bin
    """

    # convert time 
    minutes_bin  = (thour*60 /binwidth)
    binnum = round(minutes_bin)
    return pd.Series( data = [binnum, (binnum*binwidth)/60.],index = ["binnum","hourbin"])

In [ ]:
df_pairs["dt"].apply(time_to_bintime)

In [ ]:
df_pairs[["binnum","hour_bin"]]= df_pairs["dt"].apply(time_to_bintime)
df_pairs = df_pairs.astype({"binnum":"int"})

In [ ]:
.5*60/10

In [ ]:
df_pairs

In [ ]:
df_pairs["binnum"].hist(bins=NUMBEROFTIMEBINS,range=[0,50])

### home made profile plot

In [ ]:
all_bin_numbers = df_pairs["binnum"].unique()
all_bin_numbers = sorted(all_bin_numbers)

# loop on time-bins
all_df = []
for ibin in all_bin_numbers:
    df = df_pairs[df_pairs["binnum"] == ibin ]
    all_df.append(df)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,5))
for ibin in range(15):
    all_df[ibin]["dt"].hist(bins=NUMBEROFTIMEBINS,range=[0,2],ax=ax)


### boxplot

- https://wellbeingatschool.org.nz/information-sheet/understanding-and-interpreting-box-plots

- https://stackoverflow.com/questions/33328774/box-plot-with-min-max-average-and-standard-deviation

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(16,6))
sns.boxplot(x=df_pairs["binnum"],y=df_pairs["dPwv"],ax=ax,color="r",notch=False)

ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right',color="b");
ax.grid()
ax.set_ylim(-4,4)
ax.set_title(f"Box-Plot : PWV difference in pairs in bins of {TIMEBINWIDTHMIN:.0f} minutes")

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(16,6))
sns.violinplot(x=df_pairs["binnum"],y=df_pairs["dPwv"],ax=ax,color="r")

ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right',color="b");
ax.grid()
ax.set_ylim(-4,4)

In [ ]:
df_pairs

In [ ]:
assert False

## Do some old plots

In [ ]:
fig,ax  = plt.subplots(1,1,figsize=(8,5))   
ax.scatter(all_pwv,all_sigmapwv,marker='o',c='b')
ax.set_xlabel("$\overline{PWV}$ (mm)")
ax.set_ylabel("$\sigma(PWV)$ (mm)")
ax.set_title("Variability of PWV")
ax.axhspan(0, 2*sigma_repeatability, facecolor="grey", alpha=0.5,label="95 CL PWV repeatability")
ax.legend()
all_figs_to_pdf.append(fig)
plt.show()

## All night combined

### Merge all night arrays

In [ ]:
# Merge all night arrays
X = np.concatenate(all_X)   
Y = np.concatenate(all_Y)
Z = np.concatenate(all_Z)

### Plot 

#### PWV difference

In [ ]:

fig,ax = plt.subplots(1,1,figsize=(8,5))
ax.scatter(X,np.abs(Y),marker="o",s=1,c="r",alpha=1)
ax.set_xscale("log")
ax.set_xlim(0.,2.)
ax.grid()
ax.set_xlabel("$\Delta t$ (hours)")
ax.set_ylabel("$|\Delta PWV |$ (mm)")
ax.set_title("PWV difference and time difference")
all_figs_to_pdf.append(fig)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,5))
ax.hist2d(np.log10(X),np.abs(Y),bins=(50,50),range=((-2,1),(0,2)),cmap="YlOrRd",density=True);
#ax.hist2d(X,Y,bins=(50,50),range=((0,2),(0,2)),cmap="YlOrRd",density=True);
ax.grid()
#ax.set_xlabel("$\log_{10}(\Delta t/1hour)$")
ax.set_xlabel("$\log_{10}(\Delta t/1hour$)")
ax.set_ylabel("$/\log_{10}(|\Delta PWV |$ (mm))")
ax.set_title("PWV difference and time difference")
all_figs_to_pdf.append(fig)
plt.show()

### Correlation function

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,5))
ax.scatter(X,np.abs(Z),marker="o",s=1,c="r",alpha=1)
ax.set_xscale("log")
ax.grid()
ax.set_xlabel("$\Delta t$ (hours)")
ax.set_ylabel("$ (dpwv_1\cdot dpwv_2)/\sigma^2$ ")
ax.set_title("$Autocorrelation$")
ax.set_ylim(0,5.)
all_figs_to_pdf.append(fig)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,5))

ax.hist2d(np.log10(X),np.abs(Z),bins=(50,50),range=((-2,1),(0,0.1)),cmap="YlOrRd",density=True)
ax.grid()
ax.set_xlabel("$\log_{10}(\Delta t/1hour)$")
ax.set_ylabel("$ (dpwv_1\cdot dpwv_2)/\sigma^2$")
ax.set_title("Auto correlation")
all_figs_to_pdf.append(fig)
plt.show()

## Discrete correlation function

In [ ]:
for night in all_selected_nights:
    #select the night
    # Choose the kind of observation
    df_spec_night = df_spec_sel[df_spec_sel["nightObs"] == night]
    N = len( df_spec_night)
    print(night,N)
    if night == 20230131:
        break

In [ ]:
def ComputeZDCF(night,df_night_pwv_curve):
    """
    Compute the Discrete Covariance Curve with pyzdcf
    """
    # convert to seconds
    df_night_pwv_curve["t_sec_rel"] = df_night_pwv_curve.dt_midnight*3600.0

    # compute the time index (integer) in seconds
    t_sec_rel_min = df_night_pwv_curve["t_sec_rel"].min()
    df_night_pwv_curve["t_sec_abs"] = (df_night_pwv_curve["t_sec_rel"] - t_sec_rel_min).astype(int)

    # create the curve in the required format
    df_pwvc = df_night_pwv_curve[["t_sec_abs","PWV [mm]_x"]]

    # add the error on the point 
    df_pwvc = df_pwvc.assign(sig_pwv = lambda x: sigma_repeatability)

    # save the pwv curve in a file
    fn = f"pwv_curve_{night}.csv"
    ffn = os.path.join(datapath_input,fn)
    df_pwvc.to_csv(ffn, index=False,header=False)

    # parameters for the pyzdcf
    params_dcf = dict(autocf    =  True, # Autocorrelation (T) or cross-correlation (F)
              prefix            = 'acf',  # Output files prefix
              uniform_sampling  =  False, # Uniform sampling?
              omit_zero_lags    =  False,  # Omit zero lag points?
              minpts            =  20,     # Min. num. of points per bin (0 is a flag for default value of 11)
              num_MC            =  100,   # Num. of Monte Carlo simulations for error estimation
              lc1_name          =  fn,   # Name of the first light curve file
              lc2_name          =  fn    # Name of the second light curve file (required only if we do CCF)
             )

    # compute the ZDCF
    dcf_df = pyzdcf(input_dir  = datapath_input+"/" , 
                    output_dir = datapath_output+"/", 
                    intr       = False, 
                    parameters = params_dcf, 
                    sep        = ',', 
                    sparse     = 'auto', 
                    verbose    = False)
    return dcf_df


In [ ]:
df_night_pwv_curve = df_spec_night[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","ex_mjd","AIRMASS","TARGET"]]

In [ ]:
dcf_df = ComputeZDCF(night,df_night_pwv_curve)

In [ ]:
dcf_df

In [ ]:
xerr = dcf_df[["-sig(tau)","+sig(tau)"]].values.T/3600. 	
yerr = dcf_df[["-err(dcf)","+err(dcf)"]].values.T/3600.	
x = dcf_df["tau"].values/3600.
y = dcf_df["dcf"].values

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,4))
ax.errorbar(x,y,xerr=xerr,yerr=yerr,marker='o', mfc='red',linewidth=0.5,
         mec='red', ms=2, mew=2,ecolor="k",elinewidth=2,capsize=2,uplims=True, lolims=True)
ax.grid()
ax.set_ylim(-1,1)
ax.set_title(f"Discrete covariance function night {night}")
ax.set_xlabel("Time (hours)")
ax.set_ylabel("DCF (no units)")

## Compute the DCF over all nights

In [ ]:
# RESET figures
all_figs_to_pdf = []

In [ ]:
#all_pwv = []
#all_sigmapwv = []

all_T = []
all_DCF = []

# loop on nights
for night in all_selected_nights:
    #select the night
    # Choose the kind of observation
    df_spec_night = df_spec_sel[df_spec_sel["nightObs"] == night]
    df_spec_night_wp = df_spec_wp[df_spec_wp["nightObs"] == night]
    df_spec_night_np = df_spec_np[df_spec_np["nightObs"] == night]
    df_spec_night_np_f = df_spec_np_f[df_spec_np_f["nightObs"] == night]
    df_spec_night_np_b = df_spec_np_b[df_spec_np_b["nightObs"] == night]
    
    #select the variables
    df_night_pwv_curve = df_spec_night[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_wp = df_spec_night_wp[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_np_b = df_spec_night_np_b[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_np_f = df_spec_night_np_f[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]
    df_night_pwv_curve_np = df_spec_night_np[["Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","dt_midnight","AIRMASS","TARGET"]]

    tmin = df_night_pwv_curve["Time"].min()
    tmax = df_night_pwv_curve["Time"].max()

    list_of_targets = df_night_pwv_curve["TARGET"].unique()
    str_list_of_targets = "\n".join(list_of_targets)
    str_list_of_targets = "\n".join(["targets:",str_list_of_targets])
     

    # convert in hours
    df_night_pwv_curve["dt"] = (df_night_pwv_curve["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_wp["dt"] = (df_night_pwv_curve_wp["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_np["dt"] = (df_night_pwv_curve_np["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_np_f["dt"] = (df_night_pwv_curve_np_f["Time"] - tmin).dt.total_seconds()/3600.
    df_night_pwv_curve_np_b["dt"] = (df_night_pwv_curve_np_b["Time"] - tmin).dt.total_seconds()/3600.
    
    # extract statistics on pwv
    stat = df_night_pwv_curve[["PWV [mm]_x"]].describe()
    date_form = DateFormatter("%y-%m-%dT%H:%M")
    #date_form = DateFormatter("%y-%m-%dT%H")
    count = int(stat.loc["count"].values[0])
    mean = stat.loc["mean"].values[0]
    median = stat.loc["50%"].values[0]
    std = stat.loc["std"].values[0]
    textstr = "\n".join((f"count : {count}",
                     f"mean : {mean:.1f} mm",
                     f"median : {median:.1f} mm",
                     f"std : {std:.1f} mm",
                    ))


    N= len(df_night_pwv_curve)

    if N>10:
        # does the fit
        
        #x,y,yerr,n,chi2dof,xfit,yfit,slope,slope_err = MakeLineFit(df_night_pwv_curve)
        xfit, y_ard, y_ard_std, y_brr, y_brr_std, resy, y_mean = MakeLinearModelFit(df_night_pwv_curve)


        # Compute the DCF
        # convert the relative time in seconds
        df_dcf = ComputeZDCF(night,df_night_pwv_curve)
        
        
        
        #textstr2 = "\n".join((f"d(PWV/dt) : ",
        #             f"slope : {slope:.3f} mm/h",
        #             f"slope err : {slope_err:.3f} mm/h",
        #             f"chi2/ndeg : {chi2dof:.2f}",       
        #            ))

        #textstr3 = "\n".join((f"d(PWV/dt) : ",
        #             f"slope : {slope:.3f} mm/h",
        #             f"slope err : {slope_err:.3f} mm/h",  
        #            ))
    

        #stat.loc["slope","PWV [mm]_x"] = slope 
        #stat.loc["slope_err","PWV [mm]_x"] = slope_err 
        #stat.loc["chi2","PWV [mm]_x"] = chi2dof

        ##########
        # plot
        ############
        

        fig = plt.figure(figsize=(18,4),constrained_layout=True)
        gs = GridSpec(1, 2,  width_ratios=[1,1],figure=fig)
    
        ax = fig.add_subplot(gs[0])
        ax2 = fig.add_subplot(gs[1])
       
        leg=ax.get_legend()
        leg2=ax2.get_legend()
        
        ax.errorbar(x=df_night_pwv_curve.dt_midnight, y=df_night_pwv_curve["PWV [mm]_x"],yerr=df_night_pwv_curve["PWV [mm]_err_x"],fmt="+",color="r",ecolor="k",label="stat err",ms=1)
        df_night_pwv_curve_wp.plot(x="dt_midnight",y="PWV [mm]_x",c="cyan",ax=ax,marker='o',lw=0.0,grid=True,legend=leg,label="polar",ms=6,alpha=1)
        df_night_pwv_curve_np_b.plot(x="dt_midnight",y="PWV [mm]_x",c="r",ax=ax,marker='o',lw=0.0,grid=True,legend=leg,label="bright, no polar",ms=6,alpha=1)
        df_night_pwv_curve_np_f.plot(x="dt_midnight",y="PWV [mm]_x",c="g",ax=ax,marker='o',lw=0.0,grid=True,legend=leg,label="faint, no polar",ms=6,alpha=1)
        ax.set_xlabel("hours since midnight")
        ax.set_ylim(0.,15.)
        ax.set_ylabel("PWV (mm)")
        ax.legend(loc="upper right")

        ax.plot(xfit, y_ard,"g-",lw=3,label="ARDRegression")
        ax.fill_between(xfit, y_ard - 1.96 * y_ard_std,y_ard + 1.96 * y_ard_std,alpha=0.3,label=r"95% CL",facecolor='green')

        #ax.plot(xfit2, y_brr,"r-",label="BayesianRidge")
        #ax.fill_between(xfit2, y_brr - 1.96 * y_brr_std,y_brr + 1.96 * y_brr_std,alpha=0.5,label=r"95% CL",facecolor='red')
      
       
        ax.text(0.8, 0.5, str_list_of_targets, transform=ax.transAxes, fontsize=12,verticalalignment='top', bbox=props,alpha=0.5)
        ax.set_title(f"night {night}")
        ax.axvline(x=0,color="k",lw=2)


        inset_ax = inset_axes(ax,width="20%", # width = 30% of parent_bbox
                                          height="40%", # height : 1 inch
                                          loc="upper left")
        
        textstr_sigma = f"$\sigma = $" +f"{sigma:.2f} mm"
        inset_ax.hist(resy,bins=30,facecolor="b")
        inset_ax.set_xlabel("residuals (mm)")
        inset_ax.text(0.1, 0.9, textstr_sigma , transform=inset_ax.transAxes, fontsize=8,verticalalignment='top', bbox=props,alpha=1)



        ## plot DCF
   
        xerr = df_dcf[["-sig(tau)","+sig(tau)"]].values.T/3600. 	
        yerr = df_dcf[["-err(dcf)","+err(dcf)"]].values.T/3600.	
        xerr= xerr.clip(0)
        yerr= yerr.clip(0)
        
        x = df_dcf["tau"].values/3600.
        y = df_dcf["dcf"].values

        all_T.append(x)
        all_DCF.append(y)
        
        ax2.errorbar(x,y,xerr=xerr,yerr=yerr,marker='o', mfc='red',linewidth=0.5,
        mec='red', ms=2, mew=2,ecolor="k",elinewidth=2,capsize=2,uplims=True, lolims=True)
        ax2.grid()
        ax2.set_ylim(-1,1)
        ax2.set_title(f"Discrete covariance function night {night}")
        ax2.set_xlabel("Time (hours)")
        ax2.set_ylabel("DCF (no units)")


        all_figs_to_pdf.append(fig) 
        
        plt.tight_layout()
        figname =f"{pathfigs}/pwvdcf_per_night_{night}"+figtype
        plt.savefig(figname)
        plt.show()


## Summary over all DCF

- colors : https://r02b.github.io/seaborn_palettes/

In [ ]:
import seaborn as sns

In [ ]:
NNt = len(all_T)
NNd = len(all_DCF)
assert NNt == NNd

In [ ]:
NNt

In [ ]:
palette = sns.color_palette("bright",NNt)

In [ ]:
palette

In [ ]:
palette[1]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,6))
for idx in range(NNt):
    x = all_T[idx]
    y = all_DCF[idx]
    ax.plot(x,y,marker= 'o',color=palette[idx],lw=2)
ax.grid()
ax.set_xscale("log")
ax.set_title("Discrete covariance function for all nights")
ax.set_xlabel("Time (hours)")
ax.set_ylabel("DCF (no units)")

In [ ]:
all_T_flat = np.concatenate(all_T)
all_DCF_flat = np.concatenate(all_DCF)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,5))

ax.hist2d(all_T_flat,np.abs(all_DCF_flat),bins=(50,50),range=((0,2),(0,1)),cmap="YlOrRd")
ax.grid()
ax.set_xlabel("$\Delta t/1hour)$")
ax.set_ylabel("$DCF$")
ax.set_title("Auto correlation function")
all_figs_to_pdf.append(fig)
plt.show()

In [ ]:
index_sel = np.where(all_T_flat > 0 )[0]
all_T_flat = all_T_flat[index_sel]
all_DCF_flat = all_DCF_flat[index_sel]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,5))

ax.hist2d(np.log10(all_T_flat),np.abs(all_DCF_flat),bins=(50,50),range=((-2,1),(0,1)),cmap="YlOrRd")
ax.grid()
ax.set_xlabel("$\log_{10}(\Delta t/1hour)$")
ax.set_ylabel("$DCF$")
ax.set_title("Auto correlation function")
all_figs_to_pdf.append(fig)
plt.show()

## Save all figs in a pdf file

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
pdf_filename = f"holo_PWV_night_fitvariation_polydeg_{POLY_DEGREE_MAX}_timecorrelationDCF_tight.pdf"
pdf_fullfilename = os.path.join(pathfigs,pdf_filename)
with PdfPages(pdf_fullfilename) as pdf:
    for fig in all_figs_to_pdf:
        pdf.savefig(fig, bbox_inches='tight') 

In [ ]:
pdf_filename = f"holo_PWV_night_fitvariation_polydeg_{POLY_DEGREE_MAX}_timecorrelationDCF_notight.pdf"
pdf_fullfilename = os.path.join(pathfigs,pdf_filename)
with PdfPages(pdf_fullfilename) as pdf:
    for fig in all_figs_to_pdf:
        pdf.savefig(fig) 